In [15]:
import requests
from requests import Session
import pandas as pd

In [338]:
class MarketData:
    # https://docs.kucoin.com/ - API Documentaion

    def __init__(self):
        self.apiurl = "https://api.kucoin.com"  # base url for kucoin
        # self.headers = {
        #     "KC-API-SIGN": signature,
        #     "KC-API-TIMESTAMP": str(now),
        #     "KC-API-KEY": api_key,
        #     "KC-API-PASSPHRASE": passphrase,
        #     "KC-API-KEY-VERSION": "2",
        # }
        self.session = Session()

    # returns all tickers
    def getAllTickers(self):
        url = self.apiurl + "/api/v1/market/allTickers"
        r = self.session.get(url)
        data = r.json()["data"]["ticker"]
        df = pd.DataFrame.from_dict(data, orient="columns")
        return df.loc[
            :,
            [
                "symbolName",
                "buy",
                "sell",
                "changeRate",
                "changePrice",
                "high",
                "low",
                "vol",
                "last",
                "averagePrice",
            ],
        ]

    # returns a single trading pair ex. 'BTC-USDT'
    def getTicker(self, symbol):
        url = self.apiurl + "/api/v1/market/orderbook/level1"
        parameters = {"symbol": symbol}
        r = self.session.get(url, params=parameters)
        data = r.json()["data"]
        # dictionary to dataframe
        df = pd.DataFrame.from_dict(data, orient="index", columns=[symbol])
        return df

    # returns the order book for a trading pair
    def getPartOrderBook(self, symbol, amount):  # does not required API KEY
        # amount can be 20 pieces of data or 100 pieces of data
        url = self.apiurl + "/api/v1/market/orderbook/level2_" + amount
        parameters = {"symbol": symbol, "amount": amount}
        r = self.session.get(url, params=parameters)
        bids = r.json()["data"]["bids"]
        asks = r.json()["data"]["asks"]
        dfBids = pd.DataFrame.from_dict(bids, orient="columns")
        dfAsks = pd.DataFrame.from_dict(asks, orient="columns")
        dfAsks.columns = ["", "Asks"]
        dfBids.columns = ["", "Bids"]
        return dfBids, dfAsks

    # returns the full order book for a trading pair
    def getFullOrderBook(self, symbol):  # needs headers and API key
        url = self.apiurl + "/api/v3/market/orderbook/level2"
        parameters = {"symbol": symbol}
        r = self.session.get(url, params=parameters)
        data = r.json()
        return data

    # returns the fiat price of a symbol
    def getFiatPrice(self, symbol):
        url = self.apiurl + "/api/v1/prices"
        parameters = {"symbol": symbol}
        r = self.session.get(url, params=parameters)
        data = r.json()["data"][symbol]
        return data

In [339]:
kucoinAPI = MarketData()

In [340]:
# kucoinAPI.getAllTickers()

In [341]:
kucoinAPI.getTicker('ETH-USDT')

,ETH-USDT
time,1653279170647
sequence,1628191978550
price,2036.31
size,0.0032153
bestBid,2036.31
bestBidSize,3.668676
bestAsk,2036.32
bestAskSize,13.4844


In [342]:
kucoinAPI.getPartOrderBook('ETH-USDT', '20')

(                   Bids
 0   2036.31    3.668676
 1    2036.3   0.0891103
 2   2036.29      7.5067
 3   2036.06    7.170894
 4   2036.04  13.0779749
 5   2036.01   5.5812706
 6      2036   0.5877738
 7   2035.99  21.3887366
 8   2035.98   5.6404538
 9   2035.97   5.6475701
 10  2035.94   1.9915141
 11  2035.91   3.0471046
 12  2035.79   0.5999999
 13  2035.59         0.1
 14  2035.55   0.2678657
 15  2035.47   0.0361107
 16  2035.46     0.90378
 17  2035.44     0.90378
 18  2035.42   4.1586549
 19  2035.39   1.0247859,
                   Asks
 0   2036.32    13.4844
 1   2036.38  0.1229127
 2   2036.39  2.7460581
 3   2036.43  0.0005931
 4   2036.48        1.5
 5   2036.49  2.1891952
 6    2036.5  4.0067938
 7   2036.58     0.0018
 8   2036.63  2.6604464
 9   2036.65  7.2556614
 10  2036.66  3.2372549
 11  2036.67  5.2525617
 12  2036.68    2.41034
 13  2036.72  0.0023999
 14  2036.74  0.2458188
 15  2036.76  2.3077171
 16  2036.82  0.1344317
 17  2036.83  9.6431085
 18  2036.84     7

In [344]:
kucoinAPI.getFiatPrice('CO')

'2033.96042023'